In [1]:
## Convert to trn file

root = "/private/home/qiantong/tmp/sclite/convert/to_convert"
with open(root + '.txt') as fin:
    with open(root + '.ref', 'w') as fref, open(root + '.hyp', 'w') as fhyp:
        lines = fin.readlines()
        for i in range(len(lines) // 3):
            ref = lines[i * 3][5:].strip()
            hyp = lines[i * 3 + 1][4:].strip()
            
            fref.write(ref + ' ({})\n'.format(i))
            fhyp.write(hyp + ' ({})\n'.format(i))
            
            # break

In [31]:
!head /private/home/qiantong/tmp/sclite/convert/to_convert.txt

T :  is that the wind no no only two devils that blow through the murderer's ribs to and fro in the ghosts moiishine the ghosts moonshine 39 iii what dost thou strain above her lovely throat's whiteness a silken chain to cover her bosom's brightness 
P:  is that the wind no now only two devils that blow through the murderous ribs to and fro in the ghost moonshine what dost thou strain above her lovely throat whiteness a silken chain to cover her bosom's brightness

T :  only two devils that blow through the murderer's ribs to and fro in the ghosts moiishine the ghosts moonshine 39 iii what dost thou strain above her lovely throat's whiteness 
P:  only two devils that blow through the murderer's ribs to and fro in the ghost's moonshine what dost thou strain above her lovely throat whiteness

T :  is that the wind no no only two devils that blow through the murderer's ribs to and fro in the ghosts moiishine the ghosts moonshine 39 iii what dost thou strain above her lovely throat's white

In [2]:
## Run sclite

# fair cluster: /private/home/qiantong/sclite/sctk-2.4.10/bin/sclite
# fb cluster: /home/qiantong/sclite on devgpu147.prn2

!/private/home/qiantong/sclite/sctk-2.4.10/bin/sclite -r /private/home/qiantong/tmp/sclite/convert/to_convert.ref trn -h /private/home/qiantong/tmp/sclite/convert/to_convert.hyp trn -i rm -o pra

sclite: 2.10 TK Version 1.3
Begin alignment of Ref File: '/private/home/qiantong/tmp/sclite/convert/to_convert.ref' and Hyp File: '/private/home/qiantong/tmp/sclite/convert/to_convert.hyp'
Error: extract_speaker can't locate RM id (0)
    Alignment# 1 for speaker           Error: extract_speaker can't locate RM id (1)
    Alignment# 2 for speaker           Error: extract_speaker can't locate RM id (2)
    Alignment# 3 for speaker           Error: extract_speaker can't locate RM id (3)
    Alignment# 4 for speaker           Error: extract_speaker can't locate RM id (4)
    Alignment# 5 for speaker           Error: extract_speaker can't locate RM id (5)
    Alignment# 6 for speaker           Error: extract_speaker can't locate RM id (6)
    Alignment# 7 for speaker           Error: extract_speaker can't locate RM id (7)
    Alignment# 8 for speaker           Error: extract_speaker can't locate RM id (8)
    Alignment# 9 for speaker           Error: extract_speaker can't locate RM id (9)


In [19]:
## Helper functions

def is_full(word, char):
    for c in word:
        if c != char:
            return False
    return True

print(is_full('   ', ' '))
print(is_full('***', ' '))
print(is_full('***', '*'))

def is_number(word):
    if word.isnumeric():
        return True
    if word.endswith('st') or word.endswith('nd') or word.endswith('rd') or word.endswith('th'):
        return word[:-2].isnumeric()
    return False

print(is_number('123'))
print(is_number('123q'))
print(is_number('3rd'))
print(is_number('third'))

True
False
True
True
False
True
False


In [25]:
## Convert

import re

file_offset = 10
group = 6

with open('/private/home/qiantong/tmp/sclite/convert/to_convert.hyp.pra') as f:
    lines = f.readlines()
    n_groups = (len(lines) - 1 - file_offset) // group
    for i in range(n_groups):
        need_replacing = False
        
        ref = lines[file_offset + i * group + 2].strip()
        hyp = lines[file_offset + i * group + 3].strip()
        eva = lines[file_offset + i * group + 4]
        
        print('--- {} ---'.format(i))
        
        length = len(ref)
        start = 6
        eva = list(eva)
        for j in range(start, length):
            if ref[j] != ' ' and ref[j - 1] == ' ' and eva[j] == ' ':
                eva[j] = 'X'
        eva = ''.join(eva)

        #print(ref)
        #print(hyp)
        #print(eva)
        
        ref_words = re.sub(' +', ' ', ref[start:].strip()).split()
        hyp_words = re.sub(' +', ' ', hyp[start:].strip()).split()
        eval_actions = re.sub(' +', ' ', eva[start:].strip()).split()
        
        #print(ref_words)
        #print(hyp_words)
        #print(eval_actions)
        
        #print(len(ref_words))
        #print(len(hyp_words))
        #print(len(eval_actions))
        
        start = 0
        length = len(ref_words)
        final_words = ref_words.copy()
        eval_actions.append('X')
        while start < length:
            for end in range(start, length + 1):
                if eval_actions[end] == 'X':
                    break
            
            if end > start:
                has_number = False
                for j in range(start, end):
                    if is_number(ref_words[j]):
                        has_number = True
                        break
                
                if has_number:
                    has_valid_replace = False
                    for j in range(start, end):
                        if is_number(ref_words[j]) and not is_full(hyp_words[j], '*'):
                            has_valid_replace = True
                            break
                            
                    if has_valid_replace:
                        need_replacing = True
                        print('!!Replacing: {}, {}'.format(start, end))
                        for j in range(start, end):
                            if not is_full(hyp_words[j], '*'):
                                final_words[j] = hyp_words[j]
                                print(hyp_words[j])
                            else:
                                final_words[j] = ''
                
            start = end + 1
        
        final_words = [w.strip().lower() for w in final_words if len(w) > 0]
        if need_replacing:
            print(ref)
            print("CVT: ", ' '.join(final_words))
            
            #print(ref_words)
            #print(hyp_words)
            #print(eval_actions)
            #print(final_words)
            #break

--- 0 ---
--- 1 ---
--- 2 ---
--- 3 ---
!!Replacing: 24, 26
NINE
POUNDS
REF:  knowing that she is in the right she corrects me she cannot bear that a falsehood should prevail it is not a question of **** 9      it is a question of truth
CVT:  knowing that she is in the right she corrects me she cannot bear that a falsehood should prevail it is not a question of nine pounds it is a question of truth
--- 4 ---
!!Replacing: 17, 23
A
HUNDRED
AND
SIXTY
FIVE
POUNDS
!!Replacing: 39, 45
A
HUNDRED
AND
FIFTY
SIX
POUNDS
REF:  my wife states that the joneses have gone into a new flat of which the rent is * ******* *** ***** **** 165    a year now jones has told me personally that the rent of his new flat is * ******* *** ***** *** 156    a year i correct my wife
CVT:  my wife states that the joneses have gone into a new flat of which the rent is a hundred and sixty five pounds a year now jones has told me personally that the rent of his new flat is a hundred and fifty six pounds a year i correct m

In [45]:
import re
from difflib import SequenceMatcher


def is_number(word):
    if word.isnumeric():
        return True
    if word.endswith('st') or word.endswith('nd') or word.endswith('rd') or word.endswith('th'):
        return word[:-2].isnumeric()
    return False


def handle_numbers(ref, hyp):
    ref = ref.split()
    hyp = hyp.split()
    new = []
    s = SequenceMatcher(None, ref, hyp)
    
    replaced = False
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'equal' or j1 == j2:
            new += ref[i1:i2]
            continue
            
        has_number = False
        for i in range(i1, i2):
            w = ref[i1]
            if is_number(w):
                has_number = True
                break
                
        if not has_number:
            new += ref[i1:i2]
            continue
           
        
        print('{:7}   a[{}:{}] --> b[{}:{}] {!r:>8} --> {!r}\n'.format(tag, i1, i2, j1, j2, ref[i1:i2], hyp[j1:j2]))
        new += hyp[j1:j2]
        
        replaced = True

    return replaced, ' '.join(new)


        
root = "/private/home/qiantong/tmp/sclite/convert/to_convert"
with open(root + '.txt') as fin:
    with open(root + '.ref', 'w') as fref, open(root + '.hyp', 'w') as fhyp:
        lines = fin.readlines()
        for i in range(len(lines) // 3):
            print('--- {} ---'.format(i))
            ref = lines[i * 3][5:].strip()
            hyp = lines[i * 3 + 1][4:].strip()
            replaced, new = handle_numbers(ref, hyp)
            
            if not replaced:
                continue
            
            print('old:', ref)
            print('new:', new)
            
            # break
        
# if __name__ == "__main__":
#     ref = "knowing that she is in the right she corrects me she cannot bear that a falsehood should prevail it is not a question of 9 it is a question of truth"
#     hyp = "knowing that she is in the right she corrects me she cannot bear that a falsehood should prevail it is not a question of nine pounds it is a question of truth"
#     handle_numbers(ref, hyp)

--- 0 ---
--- 1 ---
--- 2 ---
replace   a[25:28] --> b[22:23] ['39', 'iii', 'what'] --> ['why']

old: is that the wind no no only two devils that blow through the murderer's ribs to and fro in the ghosts moiishine the ghosts moonshine 39 iii what dost thou strain above her lovely throat's whiteness a silken chain to cover her bosom's brightness tremble and weep not what dost thou fear
new: is that the wind no no only two devils that blow through the murderer's ribs to and fro in the ghosts moiishine the ghosts moonshine why dost thou strain above her lovely throat's whiteness a silken chain to cover her bosom's brightness tremble and weep not what dost thou fear
--- 3 ---
replace   a[24:25] --> b[24:26]    ['9'] --> ['nine', 'pounds']

old: knowing that she is in the right she corrects me she cannot bear that a falsehood should prevail it is not a question of 9 it is a question of truth
new: knowing that she is in the right she corrects me she cannot bear that a falsehood should prevai

In [20]:
truth_tsv = "/checkpoint/abaevski/data/speech/libri/1h/wav2vec/raw/train.tsv"
truth_ltr = "/checkpoint/abaevski/data/speech/libri/1h/wav2vec/raw/train.ltr"
pl_tsv = "/private/home/qiantong/w2v/data/ls_1h_rescore/train.tsv"
pl_ltr = "/private/home/qiantong/w2v/data/ls_1h_rescore/train.ltr"
concat_tsv = "/private/home/qiantong/w2v/data/ls_1h_rescore+1h_trans/train.tsv"
concat_ltr = "/private/home/qiantong/w2v/data/ls_1h_rescore+1h_trans/train.ltr"
replace_tsv = "/private/home/qiantong/w2v/data/ls959_1h_rescore+1h_trans/train.tsv"
replace_ltr = "/private/home/qiantong/w2v/data/ls959_1h_rescore+1h_trans/train.ltr"


truth_tsv = "/checkpoint/abaevski/data/speech/libri/10h/wav2vec/raw/train.tsv"
truth_ltr = "/checkpoint/abaevski/data/speech/libri/10h/wav2vec/raw/train.ltr"
pl_tsv = "/private/home/qiantong/w2v/data/ls_10h_rescore/train.tsv"
pl_ltr = "/private/home/qiantong/w2v/data/ls_10h_rescore/train.ltr"
concat_tsv = "/private/home/qiantong/w2v/data/ls_10h_rescore+10h_trans/train.tsv"
concat_ltr = "/private/home/qiantong/w2v/data/ls_10h_rescore+10h_trans/train.ltr"
replace_tsv = "/private/home/qiantong/w2v/data/ls950_10h_rescore+10h_trans/train.tsv"
replace_ltr = "/private/home/qiantong/w2v/data/ls950_10h_rescore+10h_trans/train.ltr"


truth_tsv = "/checkpoint/abaevski/data/speech/libri/100h/wav2vec/raw/train.tsv"
truth_ltr = "/checkpoint/abaevski/data/speech/libri/100h/wav2vec/raw/train.ltr"
pl_tsv = "/private/home/qiantong/w2v/data/ls_100h_rescore/train.tsv"
pl_ltr = "/private/home/qiantong/w2v/data/ls_100h_rescore/train.ltr"
concat_tsv = "/private/home/qiantong/w2v/data/ls_100h_rescore+100h_trans/train.tsv"
concat_ltr = "/private/home/qiantong/w2v/data/ls_100h_rescore+100h_trans/train.ltr"
replace_tsv = "/private/home/qiantong/w2v/data/ls860_100h_rescore+100h_trans/train.tsv"
replace_ltr = "/private/home/qiantong/w2v/data/ls860_100h_rescore+100h_trans/train.ltr"


truth_tsv = "/checkpoint/abaevski/data/speech/libri/10m/wav2vec/raw/train.tsv"
truth_ltr = "/checkpoint/abaevski/data/speech/libri/10m/wav2vec/raw/train.ltr"
pl_tsv = "/private/home/qiantong/w2v/data/ls_10m_rescore/train.tsv"
pl_ltr = "/private/home/qiantong/w2v/data/ls_10m_rescore/train.ltr"
concat_tsv = "/private/home/qiantong/w2v/data/ls_10m_rescore+10m_trans/train.tsv"
concat_ltr = "/private/home/qiantong/w2v/data/ls_10m_rescore+10m_trans/train.ltr"
replace_tsv = "/private/home/qiantong/w2v/data/ls960_10m_rescore+10m_trans/train.tsv"
replace_ltr = "/private/home/qiantong/w2v/data/ls960_10m_rescore+10m_trans/train.ltr"

In [21]:
data_dict = {}

data_base = ""

with open(pl_tsv) as ftsv, open(pl_ltr) as fltr:
    tsv_lines = ftsv.readlines()
    data_base = tsv_lines[0]
    for i, line in enumerate(fltr):
        tsv_line = tsv_lines[i+1]
        sp = tsv_line.strip().split('\t')
        key = sp[0].split('/')
        base = '/'.join(key[1:])
        
        data_dict[base] = [sp[0], sp[1], line.strip()]
        
with open(truth_tsv) as ftsv, open(truth_ltr) as fltr:
    tsv_lines = ftsv.readlines()
    for i, line in enumerate(fltr):
        tsv_line = tsv_lines[i+1]
        sp = tsv_line.strip().split('\t')
        base = sp[0]
        
        assert base in data_dict, base
        assert data_dict[base][1] == sp[1]
        data_dict[base].append(line)
        
with open(concat_tsv, 'w') as ftsv, open(concat_ltr, 'w') as fltr:
    ftsv.write(data_base)
    
    for _, v in data_dict.items():
        ftsv.write("{}\t{}\n".format(v[0], v[1]))
        fltr.write(v[2] + '\n')
        
        if len(v) == 4:
            ftsv.write("{}\t{}\n".format(v[0], v[1]))
            fltr.write(v[3] + '\n')
            
with open(replace_tsv, 'w') as ftsv, open(replace_ltr, 'w') as fltr:
    ftsv.write(data_base)
    
    for _, v in data_dict.items():
        ftsv.write("{}\t{}\n".format(v[0], v[1]))
        fltr.write(v[-1] + '\n')
        

In [31]:
import os

out_root = "/private/home/qiantong/wav2letter_experiments/w2v_pl/"

parts = [
    "ls960_10m_rescore+10m_trans",
    "ls959_1h_rescore+1h_trans",
    "ls950_10h_rescore+10h_trans",
    "ls860_100h_rescore+100h_trans",
]

tsv_file = "/private/home/qiantong/w2v/data/{}/train.tsv"
ltr_file = "/private/home/qiantong/w2v/data/{}/train.ltr"
out_file = "/private/home/qiantong/wav2letter_experiments/w2v_pl/{}.lst"


def norm_trans(line):
    line = ''.join(line.strip().split())
    return line.replace('|', ' ').lower()

for part in parts:
    with open(tsv_file.format(part)) as ftsv, open(ltr_file.format(part)) as fltr, open(out_file.format(part), 'w') as f:
        tsv_lines = ftsv.readlines()
        data_base = tsv_lines[0].strip()
        for i, line in enumerate(fltr):
            tsv_line = tsv_lines[i+1]
            sp = tsv_line.split('\t')
            duration = float(sp[-1]) / 16
            sp[-1] = str(duration)
            tsv_line = '\t'.join(sp)
            f.write('\t'.join([str(i), data_base + '/' + tsv_line.strip(), norm_trans(line)]) + '\n')
        
        
        
        

In [51]:
import random

parts = [
    '10m',
    '1h',
    '10h',
]

duration = [
    86 * 60000. * 16,
    8.6 * 3600000. * 16,
    86 * 3600000. * 16,
]

truth_root = "/checkpoint/abaevski/data/speech/libri/{}/wav2vec/raw/train"
pl_root = "/private/home/qiantong/w2v/data/ls_{}_rescore/train"
out_root = "/private/home/qiantong/w2v/data/fixed_ratio_{}_ls/train"

for j in range(3):
    part = parts[j]
    
    data_dict = {}
    data_base = ""

    with open(pl_root.format(part) + '.tsv') as ftsv, open(pl_root.format(part) + '.ltr') as fltr:
        tsv_lines = ftsv.readlines()
        data_base = tsv_lines[0]
        for i, line in enumerate(fltr):
            tsv_line = tsv_lines[i+1]
            sp = tsv_line.strip().split('\t')
            key = sp[0].split('/')
            base = '/'.join(key[1:])
            data_dict[base] = [sp[0], float(sp[1]), line.strip()]
            
    with open(truth_root.format(part) + '.tsv') as ftsv, open(truth_root.format(part) + '.ltr') as fltr:
        tsv_lines = ftsv.readlines()
        for i, line in enumerate(fltr):
            tsv_line = tsv_lines[i+1]
            sp = tsv_line.strip().split('\t')
            base = sp[0]

            assert base in data_dict, base
            assert data_dict[base][1] == float(sp[1])
            data_dict[base].append(line.strip())
            
    pl_duration = 0
    with open(out_root.format(part) + '.tsv', 'w') as ftsv, open(out_root.format(part) + '.ltr', 'w') as fltr:
        ftsv.write(data_base)
        items = list(data_dict.items())
        random.shuffle(items)
        
        for k, v in items:
            if len(v) == 3:
                if pl_duration > duration[j]:
                    continue
                pl_duration += v[1]
                    
            ftsv.write("{}\t{}\n".format(v[0], int(v[1])))
            fltr.write(v[-1] + '\n')

            
  


In [53]:
import random

parts = [
    '10m',
    '1h',
    '10h',
    '100h'
]

duration = [
    500 * 60000. * 16,
    50 * 3600000. * 16,
    500 * 3600000. * 16,
    5000 * 3600000. * 16,
]

truth_root = "/checkpoint/abaevski/data/speech/libri/{}/wav2vec/raw/train"
pl_root = "/private/home/qiantong/w2v/data/train_36s_short_{}_rescore/train"
out_root = "/private/home/qiantong/w2v/data/fixed_ratio_{}_vox/train"

for j in range(3):
    part = parts[j]
    
    data_dict = {}
    data_base = ""

    with open(pl_root.format(part) + '.tsv') as ftsv, open(pl_root.format(part) + '.ltr') as fltr:
        tsv_lines = ftsv.readlines()
        data_base = tsv_lines[0]
        for i, line in enumerate(fltr):
            tsv_line = tsv_lines[i+1]
            sp = tsv_line.strip().split('\t')
            key = sp[0].split('/')
            base = '/'.join(key[1:])
            data_dict[base] = [sp[0], float(sp[1]), line.strip()]
            
    with open(truth_root.format(part) + '.tsv') as ftsv, open(truth_root.format(part) + '.ltr') as fltr:
        tsv_lines = ftsv.readlines()
        for i, line in enumerate(fltr):
            tsv_line = tsv_lines[i+1]
            sp = tsv_line.strip().split('\t')
            base = sp[0]

            if base not in data_dict:
                data_dict[base] = [data_base.strip() + '/' + sp[0], float(sp[1]), ""]
            else:
                assert data_dict[base][1] == float(sp[1])
            data_dict[base].append(line.strip())
            
    pl_duration = 0
    with open(out_root.format(part) + '.tsv', 'w') as ftsv, open(out_root.format(part) + '.ltr', 'w') as fltr:
        ftsv.write(data_base)
        items = list(data_dict.items())
        random.shuffle(items)
        
        for k, v in items:
            if len(v) == 3:
                if pl_duration > duration[j]:
                    continue
                pl_duration += v[1]
                    
            ftsv.write("{}\t{}\n".format(v[0], int(v[1])))
            fltr.write(v[-1] + '\n')

            
  


In [42]:
outdir = "/private/home/qiantong/fl_org/experiments/chime/data/"

datadir = "/checkpoint/wav2letter/data/CHiME-5/"
train_dirs = ["train_u200k", "train_worn"]
dev_dirs = ["dev_worn", "dev_beamformit_ref"]

valid_tokens = "qwertyuiopasdfghjklzxcvbnm'-"
tokenset = {}
lexicon = {}

with open(outdir + "train_u200k_worn.withempty.lst", 'w') as fout:
    for train_dir in train_dirs:
        with open(datadir + train_dir + "/data.lst") as f:
            for line in f:
                totalcnt += 1
                sp = line.split('\t')
                text = sp[-1].strip().split()

                is_bad = False
                new_text = []
                for word in text:
                    if word[0] == '[':
                        if not word[-1] == ']':
                            print(word)
                        continue

                    if word[-1] == ']':
                        continue

                    for c in word:
                        if not c in valid_tokens:
                            print(word)
                            is_bad = True
                            break

                    new_text.append(word)

                if is_bad: #or len(new_text) == 0:
                    continue
                
                sp[-1] = ' '.join(new_text) + '\n'
                fout.write('\t'.join(sp))
#             print(text)
#             break
# print(tokenset)
# print(badcnt, totalcnt)

nîmes
nîmes
nîmes
nîmes
nîmes
#susiewigglebum
#susiewigglebum
#susiewigglebum
jalapeños
jalapeños
jalapeños
t&t
poké
poké
poké
t&t
t&t
poké
jalapeños
h&m
jalapeños
jalapeños
jalapeños
h&m
jalapeños
poké
poké
t&t
pâté
pâté
pâté
t&t
pâté
pâté
pâté
t&t
gewürztraminer
gewürztraminer
gewürztraminer
t&t
t&t
t&t
t&t
t&t
t&t
t&t
t&t
t&t
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
häagen-dazs
t&t
t&t
t&t
t&t
häagen-dazs
häagen-dazs
häagen-dazs
t&t
t&t
t&t
t&t
t&t
t&t
rosé
rosé
rosé
rosé
sautéed
sautéed
t&t
nǐ
hǎo
nǐ
hǎo
nǐ
hǎo
nǐ
hǎo
nǐ
hǎo
nǐ
hǎo
sauté
sauté
sauté
nîmes
nîmes
#susiewigglebomb
#susiewigglebomb
#susiewigglebum
#susiewigglebum
jalapeños
jalapeños
poké
poké
t&t
poké
poké
t&t
jalapeños
jalapeños
h&m
jalapeños
jalapeños
h&m
poké
poké
t&t
pâté
t&t
pâté
pâté
pâté
t&t
pâté
pâté
t&t
gewürztraminer
gewürztraminer
häagen-dazs
häagen-dazs
t&t
t&t
t&t
t&t
t&t
t&t
häagen-dazs
häagen-dazs
häagen-dazs
häage

In [21]:
lexicon = {}

hmm_words = ['mhm', 'mm', 'mmm']

with open(outdir + "train_u200k_worn.lst") as f:
    for line in f:
        sp = line.split('\t')
        text = sp[-1].strip().split()
        
        for word in text:
            clean_word = word.replace('-', ' ').strip()
            if clean_word in hmm_words:
                clean_word = 'hmm'
            clean_word = clean_word.replace(' ', '|')
            clean_word += '|'
            
            if not word in lexicon:
                lexicon[word] = [0, ""]
            lexicon[word][0] += 1
            lexicon[word][1] = ' '.join(list(clean_word))

In [22]:
valid_tokens = "qwertyuiopasdfghjklzxcvbnm'-"
for dev_dir in dev_dirs:
    with open(datadir + dev_dir + "/data.lst") as f:
        for line in f:
            sp = line.split('\t')
            text = sp[-1].strip().split()

            for word in text:
                if word[0] == '[':
                    if not word[-1] == ']':
                        print(word)
                    continue

                if word[-1] == ']':
                    continue

                for c in word:
                    if not c in valid_tokens:
                        print(word)
                        break
                        
                clean_word = word.replace('-', ' ').strip()
                if clean_word in hmm_words:
                    clean_word = 'hmm'
                clean_word = clean_word.replace(' ', '|')
                clean_word += '|'

                if not word in lexicon:
                    lexicon[word] = [0, ""]
                lexicon[word][0] += 1
                lexicon[word][1] = ' '.join(list(clean_word))

In [23]:
sorted_x = sorted(lexicon.items(), key=lambda kv: kv[1][0], reverse=True)
with open(outdir + "train+dev.lex", 'w') as fout:
    for k, v in sorted_x:
        fout.write(k + '\t' + v[1] + '\n')

In [41]:
valid_tokens = "qwertyuiopasdfghjklzxcvbnm'-"
for dev_dir in dev_dirs:
    print(dev_dir)
    totalcnt = 0
    badcnt = 0
    with open(outdir + dev_dir + ".withempty.lst", 'w') as fout:
        with open(datadir + dev_dir + "/data.lst") as f:
            for line in f:
                totalcnt += 1
                sp = line.split('\t')
                text = sp[-1].strip().split()

                is_bad = False
                new_text = []
                for word in text:
                    if word[0] == '[':
                        if not word[-1] == ']':
                            print(word)
                        continue

                    if word[-1] == ']':
                        continue

                    for c in word:
                        if not c in valid_tokens:
                            print(word)
                            is_bad = True
                            break

                    new_text.append(word)
                    
                if is_bad:# or len(new_text) == 0:
                    badcnt += 1
                    continue
                
                sp[-1] = ' '.join(new_text) + '\n'
                fout.write('\t'.join(sp))
    print(badcnt, totalcnt)

dev_worn
0 7437
dev_beamformit_ref
0 7437


In [43]:
maxlen = -1
minlen = 1e10

total_cnt = 0
long_cnt = 0
with open(outdir + "train_u200k_worn.withempty.lst") as f:
    with open(outdir + "train_u200k_worn_25s.withempty.lst", 'w') as fout:
        for line in f:
            total_cnt += 1
            sp = line.split('\t')
            duration = float(sp[-2])
            maxlen = max(maxlen, duration)
            minlen = min(minlen, duration)
            if duration > 25000:
                long_cnt += 1
                continue
            fout.write(line)
        
print(maxlen)
print(minlen)
print(long_cnt, total_cnt)

40540.0
10.0
41 359767


In [44]:
with open('/checkpoint/locronan/libri++/ctc++/train+dev-100-5000w-unk.lex') as f:
    lines = f.readlines()
    print(len(lines))

35631


In [58]:
datadir = "/checkpoint/wav2letter/data/librispeech/lists/"
lists = ['train-clean-100.lst','train-clean-360.lst','train-other-500.lst','dev-clean.lst','dev-other.lst']

word_cnt = {}

for l in lists:
    with open(datadir + l) as f:
        for line in f:
            for w in line.split()[3:]:
                if not w in word_cnt:
                    word_cnt[w] = 0
                word_cnt[w] += 1
                
datadir = "/checkpoint/wav2letter/data/librispeech/lists/"
lists = ['train-clean-100.lst','train-clean-360.lst','train-other-500.lst']

train_word_cnt = {}

for l in lists:
    with open(datadir + l) as f:
        for line in f:
            for w in line.split()[3:]:
                if not w in train_word_cnt:
                    train_word_cnt[w] = 0
                train_word_cnt[w] += 1

In [59]:
print(len(word_cnt))
print(len(train_word_cnt))

89611
89114


In [60]:
sorted_words = sorted(word_cnt.items(), key=lambda item: item[1], reverse=True)
print(sorted_words[:10])

sorted_train_words = sorted(train_word_cnt.items(), key=lambda item: item[1], reverse=True)
print(sorted_train_words[:10])

[('the', 577041), ('and', 324441), ('of', 285867), ('to', 259239), ('a', 205167), ('in', 158333), ('i', 129530), ('he', 124804), ('that', 120313), ('was', 116654)]
[('the', 570847), ('and', 320865), ('of', 282765), ('to', 256426), ('a', 202966), ('in', 156601), ('i', 128059), ('he', 123397), ('that', 119037), ('was', 115380)]


In [62]:
for n_tokens in [100, 500, 1000, 5000, 10000]:
    valid_train_words = {}
    with open('/private/home/qiantong/fl_org/experiments/sinkhorn/train960+dev-{}w-unk.tok'.format(n_tokens), 'w') as f:
        f.write("_\nUNK\n")
        for w in sorted_train_words[:n_tokens]:
            valid_train_words[w[0]] = 1
            f.write(w[0] + '\n')

    with open('/private/home/qiantong/fl_org/experiments/sinkhorn/train960+dev-{}w-unk.lex'.format(n_tokens), 'w') as f: 
        for i, w in enumerate(sorted_words):
            if w[0] not in valid_train_words:
                spell = "UNK"
            else:
                spell = w[0]
            f.write(w[0] + " _ " + spell + '\n')

In [68]:
datadir = '/checkpoint/wav2letter/data/librispeech/lists/'
trainlists = ['train-clean-100.lst', 'train-clean-360.lst', 'train-other-500.lst']

samples = {}
for tl in trainlists:
    with open(datadir + tl) as f:
        for line in f:
            sp = line.split()
            samples[sp[0]] = sp[:3]


In [81]:
modeldir = '/checkpoint/qiantong/wav2letter_experiments/sinkhorn/ksink_tds_k3_aggress1_saug2/'

with open(modeldir + 'pl.txt', 'w') as fout:
    with open(modeldir + 'raw_pl.txt') as f:
        lines = f.readlines()
        for idx in range(len(lines) // 5):
            idx = 5 * idx
            trans = lines[idx + 3].split(':')[1].replace('_ ', '').strip()
#             print(trans)
            sampleid = lines[idx + 4].split(':')[1].split(',')[0].strip()
#             print(sampleid)
            
            outtext = samples[sampleid] + [trans]
            
            fout.write('\t'.join(outtext) + '\n')
#             break

In [118]:
logtmp = "/checkpoint/qiantong/fl_experiments/benchmark/GPU_{}.log"

lineses = []
for i, ngpu in enumerate([1, 8, 16]):
    with open(logtmp.format(ngpu)) as f:
        lines = f.readlines()
        if i == 0:
            lines = [''] + lines
        lineses.append(lines)
        

for i in range(12):
    name = ' '.join(lineses[0][2 + i *13].split()[1:-1])
    if '+' in name:
        name = name.replace('+', '\t')
    else:
        name += '\tFP32'
    thrpts = []
    for lines in lineses:
        thrpts.append(lines[3 + i *13].split()[-1])
    mem = lineses[-1][12 + i *13].split()[-5]
#     print('\t'.join([name, mem] + thrpts))
    print('\t'.join([mem] + thrpts))

10.83	94.36	741.93	1481.61
7.22	268.81	2023.99	3971.88
11.59	675.66	5310.88	10524.17
5.71	942.08	7286.01	14303.12
28.78	309.80	2450.10	4880.53
14.08	558.95	4358.75	8596.04
21.57	21.86	170.10	340.04
11.33	22.89	175.01	351.20
17.78	5270.28	41842.31	81940.76
14.80	11753.74	83932.78	143284.18
16.19	182.23	1410.20	2799.94
11.73	494.39	3557.55	6715.93


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

root = '/private/home/qiantong/pseudo-labels/data/'

nthreads = 40
with open(root + 'librivox.cut.sub36s.no_label.lst') as f:
    lines = f.readlines() #[:600]
    lpt = len(lines) // nthreads + 1
    print(lpt)
    
    def proess_lines(tid):
        start = tid * lpt
        end = min(len(lines), start + lpt)
        # print(tid, start, end)
        import copy
        part_lines = copy.deepcopy(lines[start:end])
        
        with open('/scratch/qiantong/librivox.cut.sub36s.no_label.25s.lsts/{}'.format(tid), 'w') as fout:
            import sox
            for line in part_lines:
                sp = line.split()
                fn = sp[1]
                duration = sox.file_info.duration(fn) * 1000
                if duration < 25000:
                    sp[-1] = str(duration)
                    line = '\t'.join(sp).strip() + '\n'
                    fout.write(line)
                    fout.flush()
                start += 1
    
    with ThreadPoolExecutor(max_workers = nthreads) as executor:
      results = executor.map(proess_lines, range(nthreads))

402362


In [126]:
root = '/private/home/qiantong/pseudo-labels/data/'

total_duration = 0.
with open('/private/home/qiantong/push_numbers/lists/train-all.lst') as f:
    with open(root + 'librispeech_full960.no_label.25s.lst', 'w') as fout:
        for line in f:
            sp = line.split()
            duration = float(sp[2])
            if duration < 25000:
                fout.write(' '.join(sp[:3]) + '\n')
                total_duration += duration / 1000
#             print(duration)
#             break
print(total_duration / 3600)

960.9877928525109


In [39]:
ls_dict = {}

with open('/private/home/qiantong/pseudo-labels/data/librispeech_full960.no_label.25s.lst') as f:
    for line in f:
        sp = line.split()
        name = sp[0]
        idx = '-'.join(name.split('-')[-3:])
        ls_dict[idx] = line.strip()

In [40]:
with open('/private/home/qiantong/libri_1hr_hypos.tsv') as f:
    with open('/private/home/qiantong/fl_org/experiments/ema_ipl/data/libri_1hr_hypos.raw.lst', 'w') as fout:
        for line in f:
            sp = line.split('\t')
            idx = sp[0]
            try:
                detail = ls_dict[idx]
                fout.write(detail + ' ' + sp[1])
            except:
                print(idx)
                
#         print(detail)
#         break

5239-32139-0047
6951-79737-0043
5622-44585-0006
7255-291500-0005
8291-282929-0007
7357-94126-0014
7255-291500-0003
2929-85685-0079
3972-170212-0014


In [8]:
logs = ['001_log', '002_log', '003_log', '004_log', '005_log', '006_log']
for log in logs:
    with open('/checkpoint/qiantong/fl_experiments/ema_ipl/10h_ls/' + log) as f:
        with open('/checkpoint/qiantong/fl_experiments/ema_ipl/tatiana_10h/' + log, 'w') as fout:
            for line in f:
                nupdates = int(line.split('|')[1].split(':')[-1])
                if nupdates % 1500 == 0:
                    fout.write(line)

In [41]:
with open('/private/home/qiantong/fl_org/experiments/ema_ipl/data/libri_1hr_hypos.raw.lst') as f:
    with open('/private/home/qiantong/fl_org/experiments/ema_ipl/data/libri_1hr_hypos.lst', 'w') as fout:
        for line in f:
            line = line.replace('/checkpoint/antares/datasets/librispeech/audio/LibriSpeech', '/datasets01/librispeech/062419')
            fout.write(line)

In [ ]:
with open('/private/home/qiantong/pseudo-labels/data/librivox.cut.sub36s.no_label.25s.dedup.lst') as fout:
    import sox
    for line in fout:
        sp = line.split()
        fn = sp[1]
        old_duration = float(sp[-1])
        duration = sox.file_info.duration(fn) * 1000
        if abs(old_duration - duration) > 1000:
            print(line, str(duration))

In [14]:
with open('/private/home/qiantong/pseudo-labels/data/librivox.cut.sub36s.no_label.25s.dedup.lst') as f:
    with open('/private/home/qiantong/pseudo-labels/data/librivox.cut.sub36s.no_label.20s.dedup.lst', 'w') as fout:
        for line in f:
            sp = line.split()
            duration = float(sp[-1])
            if duration < 20000:
                fout.write(line)

In [51]:
name_tmp = '/checkpoint/qiantong/fl_experiments/ema_ipl/{}/{}_model_last_fixed_cache{}'

for i in range(32):
    with open(name_tmp.format('ls_dedup_10sup_lv_base_ema', '002', i), 'w') as f:
        pass

*** fi finished
*** rm-vallader finished
*** hsb finished
*** ga-IE finished
*** lt finished
*** ja finished
*** ka finished
*** lg finished
*** cnh finished
*** cv finished
*** sah finished
*** sl finished
